In [ ]:
#this program systematically identifies the POS data from the latest month that is worth the attention of sales managers and reps
#paramters to define actionable POS data designed after discussions with business development team
#looking for non-contract customers that bought an item through distribution that they also previously downloaded data from our site for
#or - looking for contract customers that bought an item they have never bought before
#end result - clean, easy to interpret data for each of our rep groups

#ideally this could be a SQL stored procedure - but the reality of the situation makes this Python script more feasible
#this has helped us make better use of POS and web activity data that was mostly just sitting untouched

In [ ]:
import pandas as pd

In [ ]:
#have these files ready beforehand and import them here
pos=pd.read_csv('August2022POS.csv',encoding='latin') #last month's POS data as generated from POS Standardization file
cad=pd.read_csv('WebAct.csv',encoding='latin') #historical activity on our website
xref=pd.read_csv('POSCustomerXRef.csv').drop_duplicates(subset=['CustomerID2']) #list of all customers
id4=pd.read_csv('4thShiftID.csv',encoding='latin').drop_duplicates(subset=['POSID']) #list of contract customers

In [ ]:
#create columns for first 8 characters of customer names in two tables - these are going to be matched up
#only want past year of web activity
#group web activity by customer, item combos - show customer email and latest activity date
pos['CStartPOS']=pos['Customer'].str[:8]
cad['DATE']=pd.to_datetime(cad['DATE'])
cad=cad[cad['DATE']>'7/31/2021'] #change
cad['CStartCAD']=cad['COMPANY NAME'].str[:8]
cad=cad.groupby(['CStartCAD','SV ITEM']).agg({'EMAIL': 'unique', 'DATE':'max'}).reset_index()

In [ ]:
#find matches
leads1=pd.merge(pos,cad,left_on=['CStartPOS','Item Number'],right_on=['CStartCAD','SV ITEM'],how='left')
leads1=leads1[leads1['CStartCAD'].notnull()]

In [ ]:
#match contract customers
id4=pd.merge(id4,xref,left_on='POSID',right_on='CustomerID2',how='inner')
id4['Customer']=id4['CustomerName_y']

In [ ]:
#show data from non-contract customers only
merged=leads1.merge(id4, how='left', left_on='Customer', right_on='Customer', indicator=True)
leads1=merged[merged['_merge']=='left_only']

In [ ]:
#show data from contract customers only
leads2=pd.merge(pos,id4,on='Customer',how='inner')

In [ ]:
#only want first-time item buys from contract customers
lpos=pd.read_csv('POS.csv',encoding='latin')
lpos=lpos[lpos['Date']<'9/1/2022'] #change
lpos=lpos[['Customer Name (Standardized)','Item Number']]
merged=leads2.merge(lpos, how='left', left_on=['Customer','Item Number'], right_on=['Customer Name (Standardized)','Item Number'], indicator=True)
leads2=merged[merged['_merge']=='left_only']

In [ ]:
#create new columns and organize tables
leads1['Direct Match']='Yes'
leads1['Existing Customer New Item']=''
leads1=leads1[['Rep Code','Direct Match','Existing Customer New Item','Date','Distributor','Customer','City','State','ZIP','Country','Item Number','Quantity','Resale Price','Total Resale','EMAIL','DATE']]
leads2['Direct Match']=''
leads2['Existing Customer New Item']='Yes'
leads2=leads2[['Rep Code','Direct Match','Existing Customer New Item','Date','Distributor','Customer','City','State','ZIP','Country','Item Number','Quantity','Resale Price','Total Resale','EMAIL','DATE','CStartPOS']]

In [ ]:
leads2=pd.merge(leads2, cad, left_on=['CStartPOS','Item Number'],right_on=['CStartCAD','SV ITEM'],how='left')
leads2.loc[leads2['CStartCAD'].notnull(), 'Direct Match'] = 'Yes'

In [ ]:
leads2=leads2[['Rep','Direct Match','Existing Customer New Item','Date','Distributor','Customer','City','State','ZIP','Country','Item Number','Quantity','Resale Price','Total Resale','EMAIL','DATE']]

In [ ]:
#tag rep name based on rep code
leads1.loc[leads1['Rep Code']==548, 'Rep'] = 'ATM'
leads1.loc[leads1['Rep Code']==564, 'Rep'] = 'Blackhart'
leads1.loc[leads1['Rep Code']==555, 'Rep'] = 'Cover 2'
leads1.loc[leads1['Rep Code']==551, 'Rep'] = 'SCS'
leads1.loc[leads1['Rep Code']==565, 'Rep'] = 'PhaseCom'
leads1.loc[leads1['Rep Code']==566, 'Rep'] = 'Vitale ESM'
leads1.loc[leads1['Rep Code']==567, 'Rep'] = 'Coastal RF'
leads1.loc[leads1['Rep Code']==331, 'Rep'] = 'NWN'
leads1.loc[leads1['Rep Code']==374, 'Rep'] = 'Wes-K'
leads1.loc[leads1['Rep Code']==401, 'Rep'] = 'House'
leads1.loc[leads1['Rep Code']==601, 'Rep'] = 'International'
leads1.loc[leads1['Rep Code']==701, 'Rep'] = 'Europe'
leads1.loc[leads1['Rep Code']==559, 'Rep'] = 'Israel'
leads1.loc[leads1['Rep Code']==630, 'Rep'] = 'India'
leads1.loc[leads1['Rep Code']==650, 'Rep'] = 'Turkey'

leads2.loc[leads2['Rep Code']==548, 'Rep'] = 'ATM'
leads2.loc[leads2['Rep Code']==564, 'Rep'] = 'Blackhart'
leads2.loc[leads2['Rep Code']==555, 'Rep'] = 'Cover 2'
leads2.loc[leads2['Rep Code']==551, 'Rep'] = 'SCS'
leads2.loc[leads2['Rep Code']==565, 'Rep'] = 'PhaseCom'
leads2.loc[leads2['Rep Code']==566, 'Rep'] = 'Vitale ESM'
leads2.loc[leads2['Rep Code']==567, 'Rep'] = 'Coastal RF'
leads2.loc[leads2['Rep Code']==331, 'Rep'] = 'NWN'
leads2.loc[leads2['Rep Code']==374, 'Rep'] = 'Wes-K'
leads2.loc[leads2['Rep Code']==401, 'Rep'] = 'House'
leads2.loc[leads2['Rep Code']==601, 'Rep'] = 'International'
leads2.loc[leads2['Rep Code']==701, 'Rep'] = 'Europe'
leads2.loc[leads2['Rep Code']==559, 'Rep'] = 'Israel'
leads2.loc[leads2['Rep Code']==630, 'Rep'] = 'India'
leads2.loc[leads2['Rep Code']==650, 'Rep'] = 'Turkey'

In [ ]:
#combine tables of two lead types; add in item descriptions
master=pd.concat([leads1,leads2],axis=0)
parts=pd.read_csv('ProductMediaUrls-9_12_2022.csv')
parts=parts[['ProductName','ProductSku']]

In [ ]:
#manipulate table to be more viewer-friendly
master=pd.merge(master,parts,left_on='Item Number',right_on='ProductSku',how='left')
master=master.drop(['ProductSku'], axis = 1)
master.rename(columns={'ProductName':'Description'},inplace=True)
master.rename(columns={'Date':'Purchase Date'},inplace=True)
master.rename(columns={'EMAIL':'Contacts'},inplace=True)
master.rename(columns={'DATE':'Web Activity Date'},inplace=True)
master=master[['Rep','Direct Match','Existing Customer New Item','Purchase Date','Distributor','Customer','City','State','ZIP','Country','Item Number','Description','Quantity','Resale Price','Total Resale','Contacts','Web Activity Date']]
master.sort_values(by=['Customer','Item Number'], inplace=True)
master['Purchase Date']=pd.to_datetime(master['Purchase Date']).dt.date
master['Web Activity Date']=pd.to_datetime(master['Web Activity Date']).dt.date

In [ ]:
#create separate leads tables for each rep
atm=master[master['Rep']=='ATM']
blackhart=master[master['Rep']=='Blackhart']
cover2=master[master['Rep']=='Cover 2']
scs=master[master['Rep']=='SCS']
phasecom=master[master['Rep']=='PhaseCom']
vitale=master[master['Rep']=='Vitale ESM']
coastalrf=master[master['Rep']=='Coastal RF']
nwn=master[master['Rep']=='NWN']

In [ ]:
#export each table for delivery to reps
atm.to_excel('C:/Users/chines/Downloads/ATM Leads.xlsx', index = False, header=True)
blackhart.to_excel('C:/Users/chines/Downloads/Blackhart Leads.xlsx', index = False, header=True)
cover2.to_excel('C:/Users/chines/Downloads/Cover 2 Leads.xlsx', index = False, header=True)
scs.to_excel('C:/Users/chines/Downloads/SCS Leads.xlsx', index = False, header=True)
phasecom.to_excel('C:/Users/chines/Downloads/PhaseCom Leads.xlsx', index = False, header=True)
vitale.to_excel('C:/Users/chines/Downloads/Vitale ESM Leads.xlsx', index = False, header=True)
coastalrf.to_excel('C:/Users/chines/Downloads/Coastal RF Leads.xlsx', index = False, header=True)
nwn.to_excel('C:/Users/chines/Downloads/NWN Leads.xlsx', index = False, header=True)

In [ ]:
#full table if desired
master.to_excel('C:/Users/chines/Downloads/8-22 Customer Leads to Reps.xlsx', index = False, header=True)